<h2 style="color:green; font-size: 40px;" align="center">PostgreSQL-PowerBI End to End Project <h2>
<h2 style="color:grey" align="center">Creating Staging Tables<h2>

In [ ]:
```sql

-- We have designed a series of tables intended to store raw data without any transformations, 
--despite naming them with the prefix "stg" (staging). These tables are structured to capture 
--data as it comes, preserving its original form for further processing and analysis.


CREATE TABLE raw.stg_cost (
    costid INTEGER PRIMARY KEY,
    year INTEGER NOT NULL,
    monthno INTEGER NOT NULL,
    countrycode CHARACTER VARYING(5) NOT NULL,
    productkey INTEGER NOT NULL,
    unitcost NUMERIC(10,2) NOT NULL,
    date DATE NOT NULL
);

CREATE TABLE raw.stg_customer (
    customerkey INTEGER PRIMARY KEY,
    geographykey INTEGER NOT NULL,
    regionkey INTEGER NOT NULL,
    businesstype CHARACTER VARYING(50) NOT NULL,
    customer CHARACTER VARYING(100) NOT NULL,
    numberemployees INTEGER NOT NULL,
    annualrevenue NUMERIC(15,2) NOT NULL,
    yearopened INTEGER NOT NULL,
    cityname CHARACTER VARYING(100) NOT NULL,
    statecode CHARACTER VARYING(10) NOT NULL,
    statename CHARACTER VARYING(100) NOT NULL,
    countrycode CHARACTER VARYING(10) NOT NULL,
    countryname CHARACTER VARYING(100) NOT NULL,
    region CHARACTER VARYING(100) NOT NULL,
    continent CHARACTER VARYING(100) NOT NULL
);

CREATE TABLE raw.stg_product (
    productkey INTEGER PRIMARY KEY, 
    productsubcategorykey INTEGER NOT NULL,
    productname CHARACTER VARYING(255) NOT NULL,
    categoryname CHARACTER VARYING(100) NOT NULL,
    subcategoryname CHARACTER VARYING(100) NOT NULL
);

CREATE TABLE raw.stg_returns (
    returnkey INTEGER PRIMARY KEY, 
    returndate DATE NOT NULL,
    orderdate DATE NOT NULL,
    salesordernumber CHARACTER VARYING(50) NOT NULL,
    customerkey INTEGER NOT NULL,
    productkey INTEGER NOT NULL,
    returnquantity INTEGER NOT NULL,
    unitprice NUMERIC(10,3) NOT NULL
);


CREATE TABLE raw.stg_sales (
    salesdetailskey INTEGER PRIMARY KEY, 
    salesheaderkey INTEGER NOT NULL,
    salesordernumber CHARACTER VARYING(50) NOT NULL,
    productkey INTEGER NOT NULL,
    productsubcategorykey INTEGER NOT NULL,
    orderquantity INTEGER NOT NULL,
    unitprice NUMERIC(10,2) NOT NULL,
    extendedamount NUMERIC(10,2) NOT NULL,
    orderdate DATE NOT NULL,
    duedate DATE NOT NULL,
    shipdate DATE NOT NULL,
    customerkey INTEGER NOT NULL,
    regionkey INTEGER NOT NULL,
    unitcost NUMERIC(10,3) NOT NULL,
    discount NUMERIC(5,4) NOT NULL
);

--These tables are essential for our data pipeline, serving as the initial repository for incoming raw data. 
--This approach ensures that we maintain the integrity and original state of the data before any processing or 
--transformation steps are applied.



<h2 style="color:green; font-size: 40px;" align="center">PostgreSQL-PowerBI <h2>
<h2 style="color:grey" align="center">Normalazing and Creating Production Tables<h2>

In [ ]:
```sql

-- Creating New Normalized Tables, starting from product table:

-- Analyzing different categories:

select distinct categoryname from raw.stg_product

-- Analyzing different subcateries:

select distinct subcategoryname from raw.stg_product

-- Normalization Method 1:
select 
	productkey, 
	productname,  
	case
		when categoryname = 'Kitchen' then 1
		when categoryname = 'Office Supply' then 2
		when categoryname = 'Clothing' then 3
		when categoryname = 'Electronic' then 4
		when categoryname = 'Personal Care' then 5
		else null 
	end as categorykey,
	categoryname,
	case
		when subcategoryname = 'Vests' then 10
		when subcategoryname = 'PC' then 20
		when subcategoryname = 'Shorts' then 30
		when subcategoryname = 'Electric Razor' then 40
		when subcategoryname = 'Android' then 50
		when subcategoryname = 'Underwear' then 60
		when subcategoryname = 'Blender' then 70
		when subcategoryname = 'Trimmer' then 80
		when subcategoryname = 'Printer' then 90
		when subcategoryname = 'Pants' then 100
		when subcategoryname = 'TV' then 110
		when subcategoryname = 'Cellphone' then 120
		when subcategoryname = 'Cofee Maker' then 130
		when subcategoryname = 'Jackets' then 140
		when subcategoryname = 'Shirts' then 150
		when subcategoryname = 'WiFi' then 160
		when subcategoryname = 'Jerseys' then 170
		when subcategoryname = 'Mac' then 180
		else null end as
	subcategorykey,
	subcategoryname,
	productsubcategorykey
from raw.stg_product

-- After manually assigning the id, you can proceed an create category and subcategory tables from the queries above
	
-- Normalization Method recommended:
-- Creating secuences:
CREATE SEQUENCE category_id
START WITH  1	
INCREMENT BY 1
MINVALUE  1 

CREATE SEQUENCE subcategory_id
START WITH 10  
INCREMENT BY 10 
MINVALUE  10  
	
-- In case you need restar or modify your secuences
ALTER SEQUENCE category_id
RESTART WITH 1  
INCREMENT BY 1  
MINVALUE 1;  

ALTER SEQUENCE subcategory_id
RESTART WITH 10  
INCREMENT BY 10
MINVALUE  10   

--Test the outcome before creating dim tables and reset que secuences:
--select categoryname, nextval('category_id') from raw.stg_product GROUP BY categoryname
--select subcategoryname, nextval('subcategory_id') from raw.stg_product GROUP BY subcategoryname 

-- Let's create dim_category	
CREATE TABLE prod.dim_category (
    category_id INT PRIMARY KEY DEFAULT nextval('category_id'),
	category_name VARCHAR(100)
);

INSERT INTO prod.dim_category (categoryname)
SELECT DISTINCT categoryname
FROM raw.stg_product;

select * from prod.dim_category

-- Let's create dim_sub_category
CREATE TABLE prod.dim_subcategory (
    subcategory_id INT PRIMARY KEY DEFAULT nextval('subcategory_id'),
	subcategory_name VARCHAR(100)
);

INSERT INTO prod.dim_subcategory (subcategoryname)
SELECT DISTINCT subcategoryname
FROM raw.stg_product;

-- Let's create dim_product

CREATE TABLE prod.dim_product (
    product_id INT PRIMARY KEY,
    product_name TEXT,
    category_id INT,
    subcategory_id INT
);

INSERT INTO prod.dim_product  (product_id, product_name, category_id, subcategory_id)
select 
	p.productkey as product_id, 
	p.productname as product_name, 
	dc.category_id, 
	dsc.subcategory_id 
	from raw.stg_product p
	left join prod.dim_category dc on dc.category_name = p.categoryname
	left join prod.dim_subcategory dsc on dsc.subcategory_name = p.subcategoryname

select * from prod.dim_product

--Let's review customers and adjust column names to follow standards and them create dim_location and dim_customers

select * from raw.stg_customer --Here is the location

CREATE TABLE prod.dim_location (
    location_id INT PRIMARY KEY,
    city VARCHAR(100),
    STATE VARCHAR(100),
    state_code VARCHAR(5),
	country VARCHAR(100),
	country_code VARCHAR(5),
	region VARCHAR(100),
	continent VARCHAR(100)
);

INSERT INTO prod.dim_location(location_id,city,state,state_code,country,country_code,region,continent)
with location as (
select 
	geographykey as location_id, 
	cityname as city,
	statename as state,
	statecode as state_code,
	countryname as country,
	countrycode as country_code,
	region,
	continent,
	row_number() over (partition by concat(cityname,statename,countryname) order by geographykey desc) as row_num   
from raw.stg_customer)
select location_id,city,state,state_code,country,country_code,region,continent from location where row_num = 1

select * from prod.dim_location

CREATE TABLE prod.dim_customer (
    customer_id INT PRIMARY KEY,
    location_id INT,
    customer_category VARCHAR(100),
    customer_name VARCHAR(100),
	fundation_year_birth INT,
	no_employees INT,
	annual_revenue NUMERIC(15, 2)
);

INSERT INTO prod.dim_customer (customer_id,location_id,customer_category,
	customer_name,fundation_year_birth,no_employees,annual_revenue)
SELECT  
	customerkey as customer_id, 
	l.location_id as location_id, 
	businesstype as customer_category, 
	customer as customer_name, 
	yearopened as fundation_year_birth,
	numberemployees as no_employees,
	annualrevenue as annual_revenue	
FROM raw.stg_customer c 
LEFT JOIN prod.dim_location l 
on c.cityname = l.city and c.statename=l.state and c.countryname =l.country;

SELECT * FROM prod.dim_customer

--let's create dim_date
--let's identify date extremes:
select 
	max(transaction_date),--2023-11-29
	min(transaction_date) -- 2020-12-29
from raw.stg_sales
	
--nos create dim_date using a serie.	
CREATE TABLE prod.dim_date as
SELECT
    date::date,
    EXTRACT(YEAR FROM date) AS year, 
    TO_CHAR(date, 'Month') AS month_name, 
    EXTRACT(MONTH FROM date) AS month_number, 
	EXTRACT(QUARTER FROM date) AS quarter,
    TO_CHAR(date, 'Day') AS day_of_week, 
    EXTRACT(DOW FROM date) AS day_of_week_number, -- (0=Sunday, 1=Monday)
	 CASE 
        WHEN EXTRACT(DOW FROM date) IN (0, 6) THEN 'Weekend'
        ELSE 'Weekday'
    END AS day_type 
FROM generate_series('2020-12-29'::date, '2023-11-29'::date, '1 day'::interval) AS date ORDER BY 4 ASC, 7 ASC;

select * from prod.dim_date

--Congrats, now we have all the dim_tables needed. 
--Please notice that dim_location can be normalized by one or 2 more dimensions. But we will stop here.

--Let's create our fact_tables

--1.1 fact_transaction
CREATE TABLE prod.fact_transaction AS
select
	salesdetailskey as tran_item_id,
	salesordernumber as order_id,
	customerkey as customer_id,
	orderdate as transaction_date,
	productkey as product_id,
	dp.category_id,
	dp.subcategory_id,
	orderquantity as order_quantity,
	unitprice as unit_price,
	extendedamount as extended_amount,
	discount as discount_rate,
	round((discount*extendedamount),2) as discount_amount,
	extendedamount-round((discount*extendedamount),2) as sale_amount,
	round(unitcost,2) as unit_cost,
	extendedamount-round((discount*extendedamount),2)-round(unitcost,2) as profit
from raw.stg_sales s
left join prod.dim_product dp on dp.product_id= s.productkey

-- Add the primary key constraint
ALTER TABLE prod.fact_transaction
ADD PRIMARY KEY (tran_item_id);

select * from prod.fact_transaction

--1.2 fact_return
CREATE TABLE prod.fact_return AS
SELECT
	returnkey as return_id,
	returndate as return_date,
	orderdate as order_date,
	salesordernumber as order_id,
	customerkey as customer_id,
	productkey as product_id,
	returnquantity as return_quantity,
	unitprice as unit_price,
	returnquantity*unitprice as return_amount
FROM raw.stg_returns;

select * from prod.fact_return

--1.2 fact_cost

select * from raw.stg_cost
	
CREATE TABLE prod.fact_cost as
SELECT
	costid as cost_id,
	productkey as product_id,
	year,
	monthno as month_no,
	countrycode as country_code,
	unitcost as unit_cost,
	date
FROM
	raw.stg_cost
;	
-- Add the primary key constraint
ALTER TABLE prod.fact_cost
ADD PRIMARY KEY (cost_id);


CREATE TABLE prod.dim_customer_life_value as

with customer_value as(
select customer_id, sum(sale_amount) as life_value_amount, count(sale_amount) as total_cus_transaction
from prod.fact_transaction group by customer_id)

select c.customer_id, cv.life_value_amount, cv.total_cus_transaction,
	case
		when total_cus_transaction >= 200 then 'Diamond'
		when total_cus_transaction between 100 and 199 then 'Silver'
		when total_cus_transaction is null then 'No Transaction'
		else 'Gold' 
	end as customer_level
from prod.dim_customer c
left join customer_value cv on c.customer_id = cv.customer_id order by total_cus_transaction desc


